# Labolatorium 2 - Kodowanie Huffmana

In [22]:
import numpy as np
import os
from collections import deque
from bitarray import bitarray
from time import process_time

## Zadanie 1 - Implementacja kodera Huffmana

### Statyczny algorytm Huffmana

In [23]:
class StaticHuffmanEncoder():
    class Node():
        def __init__(self, left=None, right=None, value=None, weight=0):
            self.left = left
            self.right = right
            self.value = value
            self.weight = weight

        def __lt__(self, other):
            return self.weight < other.weight

    def __init__(self, text):
        self.text = text
        self.freq = self.get_freq()
        self.tree = self.build_tree()
        self.codes = self.get_code()

    def get_freq(self):
        freq = {}
        for c in self.text:
            if c in freq:
                freq[c] += 1
            else:
                freq[c] = 1
        return freq
    
    def build_tree(self):
        leafs = deque(sorted([self.Node(weight=self.freq[c], value=c) for c in self.freq], key=lambda x: x.weight))
        internal_nodes = deque()

        while len(leafs) + len(internal_nodes) > 1:
            # Pop two nodes with the lowest weight
            if len(leafs) == 0:
                left = internal_nodes.popleft()
            elif len(internal_nodes) == 0:
                left = leafs.popleft()
            else:
                if leafs[0].weight < internal_nodes[0].weight:
                    left = leafs.popleft()
                else:
                    left = internal_nodes.popleft()
            if len(leafs) == 0:
                right = internal_nodes.popleft()
            elif len(internal_nodes) == 0:
                right = leafs.popleft()
            else:
                if leafs[0].weight < internal_nodes[0].weight:
                    right = leafs.popleft()
                else:
                    right = internal_nodes.popleft()
            internal_nodes.append(self.Node(left=left, right=right, weight=left.weight+right.weight))

        return internal_nodes.popleft()

        
    def get_code(self):
        codes = {}
        def traverse(node, code):
            if node.value:
                codes[node.value] = code
            else:
                traverse(node.left, code + '0')
                traverse(node.right, code + '1')
        traverse(self.tree, '')
        return codes
    
    def encode(self, text=None, encode_tree=False):
        if text is None:
            text = self.text

        encoded = bitarray()
        if encode_tree:
            encoded.extend(f'{len(self.freq):032b}')

            for c in self.codes:
                encoded.extend(f'{ord(c):08b}')
                encoded.extend(f'{self.freq[c]:032b}')

        for c in text:
            encoded.extend(self.codes[c])

        # Make coded text divisible by 8
        end_bits = 8 - len(encoded) % 8
        encoded = bitarray(f"{end_bits:08b}") + encoded + bitarray(end_bits)

        return encoded
    
    def encode_to_file(self, filename, text=None, encode_tree=False):
        encoded = self.encode(text, encode_tree)
        with open(filename, 'wb') as f:
            encoded.tofile(f)
            
        return encoded

    def decode(self, encoded: bitarray, encoded_tree = False):
        decoded = ''
        encoded = encoded[8:-int(encoded[:8].to01(),2)]

        if not encoded_tree:
            node = self.tree
            idx = 0
        else:
            no_of_codes = int(encoded[:32].to01(), 2)
            idx = 32
            for i in range(no_of_codes):
                c = chr(int(encoded[idx:idx+8].to01(), 2))
                idx += 8
                code_freq = int(encoded[idx:idx+32].to01(), 2)
                self.freq[c] = code_freq
                idx += 32

            node = self.build_tree()

        for c in encoded[idx:]:
            if not c:
                node = node.left
            else:
                node = node.right
            if node.value:
                decoded += node.value
                node = self.tree
        return decoded    
    
    def decode_from_file(self, filename, encoded_tree = False):
        with open(filename, 'rb') as f:
            encoded = bitarray()
            encoded.fromfile(f)
        return self.decode(encoded, encoded_tree)
    
    def print_tree(self):
        def traverse(node, code):
            if node.value:
                print(f"{node.value}: {code}")
            else:
                traverse(node.left, code + '0')
                traverse(node.right, code + '1')
        traverse(self.tree, '')

#### Przykład kodowania tekstu bez zapisu liczby wystąpień znaków do pliku


In [24]:
text = "Hello World!"
encoder = StaticHuffmanEncoder(text)
encoded = encoder.encode(encode_tree=False)
decoded = encoder.decode(encoded, encoded_tree=False)

print("Original text: ", text)
print("Encoded text: ", encoded)
print("Decoded text: ", decoded)

Original text:  Hello World!
Encoded text:  bitarray('000000110110011110100011001101001110101111010010')
Decoded text:  Hello World!


#### Przykład kodowania tekstu z zapisem liczby wystąpień znaków do pliku


In [25]:
text = "Hello World!"
encoder = StaticHuffmanEncoder(text)
encoded = encoder.encode(encode_tree=True)
decoded = encoder.decode(encoded, encoded_tree=True)

print("Original text: ", text)
print("Encoded text: ", encoded)
print("Decoded text: ", decoded)

Original text:  Hello World!
Encoded text:  bitarray('00000011000000000000000000000000000010010110111100000000000000000000000000000010001000010000000000000000000000000000000101001000000000000000000000000000000000010110010100000000000000000000000000000001011011000000000000000000000000000000001100100000000000000000000000000000000000010101011100000000000000000000000000000001011100100000000000000000000000000000000101100100000000000000000000000000000000010110011110100011001101001110101111010110')
Decoded text:  Hello World!


### Dynamiczny algorytm Huffmana

In [26]:
class AdaptiveHuffmanEncoder():
    class Node():
        def __init__(self, right=None, parent=None, weight=0, index=0, value=None, left=None):
            self.left = left
            self.right = right
            self.parent = parent
            self.value = value
            self.weight = weight
            self.index = index

    def __init__(self):
        self.index = 999999 # Should be a large number
        tmp = self.Node(weight=0, index=self.index, value="NYT")
        self.root = tmp
        self.nodes = {"NYT": self.root}
        self.weights = {0: set([self.root]), 1: set()}

    def add_char_to_tree(self, char):
        node = self.nodes["NYT"]
        node.left = self.Node(weight=0, index=self.index - 2, parent=node, value="NYT")
        self.weights[0].add(node.left)
        self.nodes["NYT"] = node.left

        node.right = self.Node(weight=1, index=self.index-1, parent=node, value=char)
        self.weights[1].add(node.right)
        self.nodes[char] = node.right
        
        node.value = None
        self.index -= 2

        self.increment(node)

    def increment(self, node):
        while node != self.root:
            node = node.parent
            block_leader = max(self.weights[node.weight], key=lambda x: x.index)

            if node != block_leader:
                # Swap nodes
                node.index, block_leader.index = block_leader.index, node.index

                # If nodes have same parent, swap subtrees
                if node.parent == block_leader.parent:
                    if node == node.parent.left:
                        node.parent.right = node
                        node.parent.left = block_leader
                    else:
                        node.parent.right = block_leader
                        node.parent.left = node
                else:
                    # Swap nodes
                    if node == node.parent.left:
                        node.parent.left = block_leader
                    else:
                        node.parent.right = block_leader

                    if block_leader.parent.left == block_leader:
                        block_leader.parent.left = node
                    else:
                        block_leader.parent.right = node
                    
                    block_leader.parent, node.parent = node.parent, block_leader.parent

            # Update weights
            self.weights[node.weight].remove(node)
            node.weight += 1

            if node.weight not in self.weights:
                self.weights[node.weight] = set([node])
            else:
                self.weights[node.weight].add(node)

    def get_code(self, char):
        node = self.nodes[char]
        code = ""
        while node != self.root:
            if node == node.parent.left:
                code += "0"
            else:
                code += "1"
            node = node.parent

        return bitarray(code[::-1])

    def encode(self, text):
        coded_text = bitarray()
        for char in text:
            if char in self.nodes:
                coded_text += self.get_code(char)
                self.increment(self.nodes[char])
            else:
                # Encode char as 8 bits (we add # code because we need to know when to decode first occurence of char)
                coded_text += self.get_code("NYT") + bitarray(f"{ord(char):08b}")
                self.add_char_to_tree(char)
        
        # Make coded text divisible by 8
        end_bits = 8 - len(coded_text) % 8
        coded_text = bitarray(f"{end_bits:08b}") + coded_text + bitarray(end_bits)
        return coded_text
    
    
    def encode_to_file(self, text, filename):
        coded_text = self.encode(text)
        with open(filename, 'wb') as f:
            coded_text.tofile(f)
        return coded_text

        
    @staticmethod
    def decode(encoded_text):
        tree = AdaptiveHuffmanEncoder()
        bit = 0
        encoded = encoded_text[8:-int(encoded_text[:8].to01(),2)]
        decoded = ""
        node = tree.root
        
        while bit < len(encoded):
            # Get char from tree
            while not (node.left is None and node.right is None):
                if not encoded[bit]:
                    node = node.left
                else:
                    node = node.right
                bit += 1
            if node.value == "NYT":
                # Read 8 bits and add new node to tree
                decoded_char = chr(int(encoded[bit:bit + 8].to01(), 2))
                tree.add_char_to_tree(decoded_char)
                bit += 8
            else:
                decoded_char = node.value
                tree.increment(tree.nodes[decoded_char])
                
            node = tree.root
            decoded += decoded_char

        return decoded
    
    @staticmethod
    def decode_from_file(filename):
        with open(filename, 'rb') as f:
            encoded = bitarray()
            encoded.fromfile(f)
        return AdaptiveHuffmanEncoder.decode(encoded)
    
    def print_tree(self):
        def traverse(node, code):
            if node.value:
                print(f"{node.value}: {code}")
            else:
                traverse(node.left, code + '0')
                traverse(node.right, code + '1')
        traverse(self.root, '')

#### Przykład kodowania tekstu 


In [27]:
text = "Hello World!"
encoder = AdaptiveHuffmanEncoder()
encoded = encoder.encode(text)
decoded = AdaptiveHuffmanEncoder.decode(encoded)

print("Original text: ", text)
print("Encoded text: ", encoded)
print("Decoded text: ", decoded)

Original text:  Hello World!
Encoded text:  bitarray('000001000100100000110010100011011000011000110111100000100000100001010111011110000111001001100000110010001000001000010010')
Decoded text:  Hello World!


## Zadanie 2 - Analiza algorytmów - pomiar czasu oraz wyznaczenie współczynnika kompresji

In [7]:
def compression_ratio(read_file, write_file):
    original_size = os.path.getsize(read_file)
    coded_size = os.path.getsize(write_file)
    return (1 - coded_size / original_size)*100

def test_huffman(filepath):
    text = open(filepath, 'r').read()
    
    filename = filepath.split("/")[-1]
    
    start_time = process_time()
    encoder = StaticHuffmanEncoder(text)
    encoded = encoder.encode_to_file(f"./output/compressed-static-{filename}")
    encoding_time = process_time() - start_time

    start_time = process_time()
    decoded = encoder.decode_from_file(f"./output/compressed-static-{filename}")
    decoding_time = process_time() - start_time

    if decoded != text:
        print("Static Huffman failed")
        return

    print(f"Static Huffman encoding time: {encoding_time}s")
    print(f"Static Huffman decoding time: {decoding_time}s")
    print(f"Static Huffman overall time: {encoding_time + decoding_time}s")
    print(f"Compression ratio: {compression_ratio(filepath, f'./output/compressed-static-{filename}')}%")

    start_time = process_time()
    encoder = StaticHuffmanEncoder(text)
    encoded = encoder.encode_to_file(f"./output/compressed-static-enctree-{filename}", encode_tree=True)
    encoding_time = process_time() - start_time

    start_time = process_time()
    decoded = encoder.decode_from_file(f"./output/compressed-static-enctree-{filename}", encoded_tree=True)
    decoding_time = process_time() - start_time

    if decoded != text:
        print("Static Huffman failed")
        return

    print(f"\nStatic Huffman (with tree encoded) encoding time: {encoding_time}s")
    print(f"Static Huffman (with tree encoded) decoding time: {decoding_time}s")
    print(f"Static Huffman (with tree encoded) overall time: {encoding_time + decoding_time}s")
    print(f"Compression ratio: {compression_ratio(filepath, f'./output/compressed-static-enctree-{filename}')}%")


    start_time = process_time()
    encoder = AdaptiveHuffmanEncoder()
    encoded = encoder.encode_to_file(text, f"./output/compressed-adaptive-{filename}")
    encoding_time = process_time() - start_time

    start_time = process_time()
    decoded = AdaptiveHuffmanEncoder.decode_from_file(f"./output/compressed-adaptive-{filename}")
    decoding_time = process_time() - start_time

    if decoded != text:
        print("Adaptive Huffman failed")
        return

    print(f"\nAdaptive Huffman encoding time: {encoding_time}s")
    print(f"Adaptive Huffman decoding time: {decoding_time}s")
    print(f"Adaptive Huffman overall time: {encoding_time + decoding_time}s")
    print(f"Compression ratio: {compression_ratio(filepath, f'./output/compressed-adaptive-{filename}')}%")


### Testy pliku z serwisu Project Gutenberg

In [8]:
test_huffman("./files/gutenberg_1kB.txt")

Static Huffman encoding time: 0.0007569999999998966s
Static Huffman decoding time: 0.0007760000000001099s
Static Huffman overall time: 0.0015330000000000066s
Compression ratio: 41.11328125%

Static Huffman (with tree encoded) encoding time: 0.0005389999999998452s
Static Huffman (with tree encoded) decoding time: 0.0005619999999999514s
Static Huffman (with tree encoded) overall time: 0.0011009999999997966s
Compression ratio: 12.40234375%

Adaptive Huffman encoding time: 0.005692999999999948s
Adaptive Huffman decoding time: 0.0045119999999998495s
Adaptive Huffman overall time: 0.010204999999999798s
Compression ratio: 30.56640625%


In [9]:
test_huffman("./files/gutenberg_10kB.txt")

Static Huffman encoding time: 0.0027680000000001037s
Static Huffman decoding time: 0.004329000000000027s
Static Huffman overall time: 0.007097000000000131s
Compression ratio: 41.455078125%

Static Huffman (with tree encoded) encoding time: 0.0022800000000000598s
Static Huffman (with tree encoded) decoding time: 0.0039010000000001543s
Static Huffman (with tree encoded) overall time: 0.006181000000000214s
Compression ratio: 37.607421875%

Adaptive Huffman encoding time: 0.050602000000000036s
Adaptive Huffman decoding time: 0.04738500000000001s
Adaptive Huffman overall time: 0.09798700000000005s
Compression ratio: 29.19921875%


In [10]:
test_huffman("./files/gutenberg_100kB.txt")

Static Huffman encoding time: 0.019571000000000005s
Static Huffman decoding time: 0.03801800000000011s
Static Huffman overall time: 0.05758900000000011s
Compression ratio: 41.80273437499999%

Static Huffman (with tree encoded) encoding time: 0.018896000000000024s
Static Huffman (with tree encoded) decoding time: 0.03863899999999987s
Static Huffman (with tree encoded) overall time: 0.05753499999999989s
Compression ratio: 41.32519531249999%

Adaptive Huffman encoding time: 0.5276419999999999s
Adaptive Huffman decoding time: 0.5248780000000002s
Adaptive Huffman overall time: 1.0525200000000001s
Compression ratio: 29.3232421875%


In [11]:
test_huffman("./files/gutenberg_1MB.txt")

Static Huffman encoding time: 0.19134699999999993s
Static Huffman decoding time: 0.3875740000000003s
Static Huffman overall time: 0.5789210000000002s
Compression ratio: 43.10178756713867%

Static Huffman (with tree encoded) encoding time: 0.18975699999999973s
Static Huffman (with tree encoded) decoding time: 0.3833359999999999s
Static Huffman (with tree encoded) overall time: 0.5730929999999996s
Compression ratio: 43.05133819580078%

Adaptive Huffman encoding time: 5.998763s
Adaptive Huffman decoding time: 5.8563790000000004s
Adaptive Huffman overall time: 11.855142s
Compression ratio: 29.991722106933594%


### Testy pliku z kodem źródłowym jądra Linuksa

In [12]:
test_huffman("./files/linux_1kB.txt")

Static Huffman encoding time: 0.0012310000000006482s
Static Huffman decoding time: 0.0008730000000003457s
Static Huffman overall time: 0.002104000000000994s
Compression ratio: 34.66796875%

Static Huffman (with tree encoded) encoding time: 0.00079200000000057s
Static Huffman (with tree encoded) decoding time: 0.0006350000000008293s
Static Huffman (with tree encoded) overall time: 0.0014270000000013994s
Compression ratio: 0.09765625%

Adaptive Huffman encoding time: 0.005613999999999564s
Adaptive Huffman decoding time: 0.005035999999998708s
Adaptive Huffman overall time: 0.010649999999998272s
Compression ratio: 21.6796875%


In [13]:
test_huffman("./files/linux_10kB.txt")

Static Huffman encoding time: 0.0029639999999986344s
Static Huffman decoding time: 0.004770000000000607s
Static Huffman overall time: 0.0077339999999992415s
Compression ratio: 29.472656249999996%

Static Huffman (with tree encoded) encoding time: 0.0023529999999993834s
Static Huffman (with tree encoded) decoding time: 0.0046160000000003976s
Static Huffman (with tree encoded) overall time: 0.006968999999999781s
Compression ratio: 25.234374999999996%

Adaptive Huffman encoding time: 0.05283599999999922s
Adaptive Huffman decoding time: 0.05039099999999941s
Adaptive Huffman overall time: 0.10322699999999863s
Compression ratio: 24.248046874999996%


In [14]:
test_huffman("./files/linux_100kB.txt")

Static Huffman encoding time: 0.020839000000000496s
Static Huffman decoding time: 0.04429100000000119s
Static Huffman overall time: 0.06513000000000169s
Compression ratio: 30.537109375000004%

Static Huffman (with tree encoded) encoding time: 0.020497000000000654s
Static Huffman (with tree encoded) decoding time: 0.043963999999999004s
Static Huffman (with tree encoded) overall time: 0.06446099999999966s
Compression ratio: 30.0888671875%

Adaptive Huffman encoding time: 0.5373599999999996s
Adaptive Huffman decoding time: 0.497088999999999s
Adaptive Huffman overall time: 1.0344489999999986s
Compression ratio: 25.386718750000004%


In [15]:
test_huffman("./files/linux_1MB.txt")

Static Huffman encoding time: 0.20204299999999975s
Static Huffman decoding time: 0.4528630000000007s
Static Huffman overall time: 0.6549060000000004s
Compression ratio: 30.16042709350586%

Static Huffman (with tree encoded) encoding time: 0.2003939999999993s
Static Huffman (with tree encoded) decoding time: 0.4510240000000003s
Static Huffman (with tree encoded) overall time: 0.6514179999999996s
Compression ratio: 30.11474609375%

Adaptive Huffman encoding time: 6.3234379999999994s
Adaptive Huffman decoding time: 6.391002s
Adaptive Huffman overall time: 12.71444s
Compression ratio: 24.505138397216797%


### Testy pliku ze znakami losowanymi z rozkładu jednostajnego

In [21]:
test_huffman("./files/uniform_1kB.txt")

Static Huffman encoding time: 0.0019540000000048963s
Static Huffman decoding time: 0.0012380000000007385s
Static Huffman overall time: 0.0031920000000056348s
Compression ratio: 34.375%

Static Huffman (with tree encoded) encoding time: 0.0015169999999997685s
Static Huffman (with tree encoded) decoding time: 0.001548999999997136s
Static Huffman (with tree encoded) overall time: 0.0030659999999969045s
Compression ratio: -81.73828125%

Adaptive Huffman encoding time: 0.013478999999996688s
Adaptive Huffman decoding time: 0.01163599999999576s
Adaptive Huffman overall time: 0.02511499999999245s
Compression ratio: 8.59375%


In [17]:
test_huffman("./files/uniform_10kB.txt")

Static Huffman encoding time: 0.002601999999999549s
Static Huffman decoding time: 0.0043500000000022965s
Static Huffman overall time: 0.006952000000001846s
Compression ratio: 33.388671875%

Static Huffman (with tree encoded) encoding time: 0.0021529999999998495s
Static Huffman (with tree encoded) decoding time: 0.004673000000000371s
Static Huffman (with tree encoded) overall time: 0.006826000000000221s
Compression ratio: 20.947265625%

Adaptive Huffman encoding time: 0.05784099999999981s
Adaptive Huffman decoding time: 0.05519899999999822s
Adaptive Huffman overall time: 0.11303999999999803s
Compression ratio: 30.683593750000004%


In [18]:
test_huffman("./files/uniform_100kB.txt")

Static Huffman encoding time: 0.01557600000000292s
Static Huffman decoding time: 0.0402780000000007s
Static Huffman overall time: 0.05585400000000362s
Compression ratio: 33.1767578125%

Static Huffman (with tree encoded) encoding time: 0.015533999999998827s
Static Huffman (with tree encoded) decoding time: 0.04001300000000185s
Static Huffman (with tree encoded) overall time: 0.05554700000000068s
Compression ratio: 31.9326171875%

Adaptive Huffman encoding time: 0.5117159999999998s
Adaptive Huffman decoding time: 0.47943599999999975s
Adaptive Huffman overall time: 0.9911519999999996s
Compression ratio: 32.86914062500001%


In [19]:
test_huffman("./files/uniform_1MB.txt")

Static Huffman encoding time: 0.1670909999999992s
Static Huffman decoding time: 0.4073330000000013s
Static Huffman overall time: 0.5744240000000005s
Compression ratio: 33.37831497192383%

Static Huffman (with tree encoded) encoding time: 0.15088599999999985s
Static Huffman (with tree encoded) decoding time: 0.40606000000000364s
Static Huffman (with tree encoded) overall time: 0.5569460000000035s
Compression ratio: 33.25681686401367%

Adaptive Huffman encoding time: 5.355218999999998s
Adaptive Huffman decoding time: 5.491534999999999s
Adaptive Huffman overall time: 10.846753999999997s
Compression ratio: 33.31718444824219%


## Wnioski

Dla większości plików, lepszy współczynnik kompresji miał algorytm statycznego kodowania Huffmana od dynamicznego kodowania Huffmana. Trochę gorszy współczynnik miał statyczny algorytm Huffmana z kodowaniem wystąpień, lecz ta obserwacja jest zrozumiała, ze względu na dodatkowe bity z liczbą wystąpień liter. Również algorytm statycznego kodowania był szybszy od dynamicznego kodowania. Ciekawą obserwacją jest wynik dla pliku uniform_1kB.txt, gdzie plik uzyskany za pomocą statycznego kodowania Huffmana z zapisem wystąpień do plików przyjął ujemny współczynnik kompresji co oznacza, że plik stał się większy po kompresji. W tym przypadku lepszym pomysłem jest skorzystanie z dynamicznego kodowania.

Zaobserwowane wyniki są związane ze sposobem budowy drzewa Huffmana. Statyczny algorytm buduje drzewo na podstawie częstości występowania symboli, natomiast dynamiczny algorytm buduje drzewo dynamicznie, na podstawie kolejności symboli w pliku. To oznacza, że kody dla symboli o małej częstości występowania mogą być dłuższe w przypadku algorytmu dynamicznego kodowania Huffmana. Dodatkowo, w przypadku dynamicznego kodowania Huffmana, musimy zapamiętywać kiedy dany symbol wystąpił po raz pierwszy, aby móc go dodać do drzewa, co może powodować wydłużenie się zakodowanego tekstu.

## Inne algorytmy kompresji

https://neptune.ai/blog/lossless-data-compression-using-arithmetic-encoding-in-python-and-its-applications-in-deep-learning